In [58]:
// dimensiones arena 
let anchoArena = 700.0
let altoArena = 1200.0

// distribución de los bloques
let filasBloques = 8
let columnasBloques = 14

// anchura de cada bloque: 700/14 = 50, los 14 bloques ocupan todo el ancho de la arena
let LX = 50.0

// altura de cada bloque: (1200/8)/6 = 25, los 8 bloques ocupan una sexta parte de la altura de la arena
let LY = 25.0

// los bloques estan entre la altura 800 y la altura 1000; limite inferior inclusivo, limite superior exclusivo [800, 1000)
let yInicial = 800.0
let yFinal = 1000.0

In [59]:
type Bloques =
    {
        Estado: Map<int*int, bool>
    }


// se almacena el estado de los bloques en un mapa cuyas claves son los indices (fila, columna) de cada bloque
// y cuyos valores son un booleano que indica si el bloque esta activo o no 
let inicializarBloques (filas: int) (columnas: int) : Bloques =
    let indices = [for x in 0 .. filas - 1 do for y in 0 .. columnas - 1 -> (x, y)]
    let estado = indices |> List.map (fun (x, y) -> ((x, y), true)) |> Map.ofList
    { Estado = estado }
    

In [60]:
// verifica que las coordenadas esten dentro de la región donde existen bloques
let validarCoordenadas (x: float) (y: float): bool =
    x >= 0.0 && x < anchoArena && y >= yInicial && y < yFinal

// A partir de las coordenadas (x,y) de la bolita, devuelve los indices del unico bloque con el cual la bolita podria colisionar
let obtenerIdxBloque (x: float) (y: float): int*int =
    let indiceFila = int ((y - yInicial) / LY)
    let indiceColumna = int (x / LX)
    (indiceFila, indiceColumna)

// obtiene las coordenadas de la esquina inferior izquierda del bloque a partir de sus indices
let obtenerCoordenadas (fila: int) (columna: int): float*float =
    let x = (float columna) * LX
    let y = yInicial + (float fila) * LY
    (x, y)

let testIdentificarBloque (x: float) (y: float): unit =
    match validarCoordenadas x y with
    | false -> printfn "No hay bloques en estas coordenadas"
    | true ->
        let (fila, columna) = obtenerIdxBloque x y
        if fila >= 0 && fila < filasBloques && columna >= 0 && columna < columnasBloques then
            printfn $"Las coordenadas corresponden al bloque ({fila}, {columna})"
        else
            printfn "Los indices obtenidos estan fuera de rango"

let testObtenerCoordenadas  (filasBloques) (columnasBloques) =
    let indices = [for x in 0 .. filasBloques - 1 do for y in 0 .. columnasBloques - 1 -> (x, y)]
    indices 
    |> List.map (fun t -> (t, obtenerCoordenadas (fst t) (snd t)))
    |> List.iter (fun (t1, t2) -> printfn $"El bloque {t1}, tiene su esquina inferior izquierda en x={fst t2}, y={snd t2}")   

In [61]:
testIdentificarBloque 0.0 800.0
testIdentificarBloque 0.0 1000.0
testIdentificarBloque 0.0 999.999
testIdentificarBloque 700.0 999.999
testIdentificarBloque 699.999 800.0
testIdentificarBloque 699.999 1000.0
testIdentificarBloque 699.999 999.999

Las coordenadas corresponden al bloque (0, 0)
No hay bloques en estas coordenadas
Las coordenadas corresponden al bloque (7, 0)
No hay bloques en estas coordenadas
Las coordenadas corresponden al bloque (0, 13)
No hay bloques en estas coordenadas
Las coordenadas corresponden al bloque (7, 13)


In [62]:
testObtenerCoordenadas 8 14

El bloque (0, 0), tiene su esquina inferior izquierda en x=0, y=800
El bloque (0, 1), tiene su esquina inferior izquierda en x=50, y=800
El bloque (0, 2), tiene su esquina inferior izquierda en x=100, y=800
El bloque (0, 3), tiene su esquina inferior izquierda en x=150, y=800
El bloque (0, 4), tiene su esquina inferior izquierda en x=200, y=800
El bloque (0, 5), tiene su esquina inferior izquierda en x=250, y=800
El bloque (0, 6), tiene su esquina inferior izquierda en x=300, y=800
El bloque (0, 7), tiene su esquina inferior izquierda en x=350, y=800
El bloque (0, 8), tiene su esquina inferior izquierda en x=400, y=800
El bloque (0, 9), tiene su esquina inferior izquierda en x=450, y=800
El bloque (0, 10), tiene su esquina inferior izquierda en x=500, y=800
El bloque (0, 11), tiene su esquina inferior izquierda en x=550, y=800
El bloque (0, 12), tiene su esquina inferior izquierda en x=600, y=800
El bloque (0, 13), tiene su esquina inferior izquierda en x=650, y=800
El bloque (1, 0), t

In [63]:
// funciones para el evento de colisión con un bloque

// verifica si hay un bloque activo en las coordenadas (x,y)
let hayBloque (x: float) (y: float) (bloques: Bloques) : bool =
    match validarCoordenadas x y with
    | false -> false
    | true ->
        let (indiceFila, indiceColumna) = obtenerIdxBloque x y
        let estado = bloques.Estado |> Map.find (indiceFila, indiceColumna)
        estado


let testHayBloque (x: float) (y: float) (bloques: Bloques) : unit =
    match hayBloque x y bloques with
    | true -> printfn "Hay un bloque en las coordenadas x=%f, y=%f" x y
    | false -> printfn "No hay un bloque en las coordenadas x=%f, y=%f" x y


// cambia el estado de un bloque a false
let desactivarBloque (fila: int) (columna: int) (bloques: Bloques) : Bloques=
    let nuevoEstado = bloques.Estado |> Map.add (fila, columna) false
    { bloques with Estado = nuevoEstado}


let colisionConBloque (x: float) (y: float) (bloques: Bloques) : Bloques =
    match hayBloque x y bloques with
    | false -> bloques
    | true ->
        let (indiceFila, indiceColumna) = obtenerIdxBloque x y
        let nuevosBloques = desactivarBloque indiceFila indiceColumna bloques
        nuevosBloques        

In [64]:
let testX = 0.0
let testY = 800.0
let testBloques = inicializarBloques filasBloques columnasBloques
let (idxFila, idxColumna) = obtenerIdxBloque testX testY

printfn $"Las coordenadas x={testX}, y={testY} corresponden al bloque ({idxFila}, {idxColumna})"
printfn "Bloques antes de la colision:"
printfn "%A" testBloques
testHayBloque testX testY testBloques
let newBloques = colisionConBloque testX testY testBloques
printfn "Bloques despues de la colision:"
printfn "%A" newBloques
testHayBloque testX testY newBloques

Las coordenadas x=0, y=800 corresponden al bloque (0, 0)
Bloques antes de la colision:
{ Estado =
   map
     [((0, 0), true); ((0, 1), true); ((0, 2), true); ((0, 3), true);
      ((0, 4), true); ((0, 5), true); ((0, 6), true); ((0, 7), true);
      ((0, 8), true); ...] }
Hay un bloque en las coordenadas x=0.000000, y=800.000000
Bloques despues de la colision:
{ Estado =
   map
     [((0, 0), false); ((0, 1), true); ((0, 2), true); ((0, 3), true);
      ((0, 4), true); ((0, 5), true); ((0, 6), true); ((0, 7), true);
      ((0, 8), true); ...] }
No hay un bloque en las coordenadas x=0.000000, y=800.000000


In [48]:
type Choque_con_bloque = 
        | Vertical
        | Horizontal

let Tipo_choque (bolita:bolita.Bolita) (fila: int) (columna: int) =
        let bloque_x, bloque_y = Bloques.obtenerCoordenadas fila columna
        let distancias_a_bloque = [bolita.y - bloque_y; bloque_y + Bloques.LY - bolita.y; bolita.x - bloque_x; bloque_x + Bloques.LX - bolita.x]
        let minimo = List.min distancias_a_bloque
        if minimo = bolita.y - bloque_y || minimo = bloque_y + Bloque.LY - bolita.y then
            Vertical
        else
            Horizontal

let Interaccion_bloques (estado_bloques: Bloques.Bloques) (bolita: bolita.Bolita) =
    match hayBloque bolita.x bolita.y estado_bloques with
    | false -> 
        (bolita, estado_bloques)
    | true ->
        let fila, columna = obtenerIdxBloque bolita.x bolita.y
        let nuevosBloques = desactivarBloque fila columna estado_bloques
        let tipo_choque = Tipo_choque bolita fila columna
        match tipo_choque with
        | Vertical -> ({ bolita with vy = -bolita.vy }, nuevosBloques)
        | Horizontal -> ({ bolita with vx = -bolita.vx }, nuevosBloques)

Error: input.fsx (5,25)-(5,31) typecheck error The namespace or module 'bolita' is not defined.
input.fsx (6,34)-(6,41) typecheck error The value, namespace, type or module 'Bloques' is not defined.
input.fsx (7,36)-(7,44) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (7,68)-(7,75) typecheck error The value, namespace, type or module 'Bloques' is not defined. Maybe you want one of the following:
   bloque_x
   bloque_y
input.fsx (7,81)-(7,89) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (7,91)-(7,99) typecheck error Lookup on object of indeterminate type based on information